# Credit Risk Resampling Techniques

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [17]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [18]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(15)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
5,25000.0,0.1797,634.43,MORTGAGE,90000.0,Source Verified,Mar-2019,low_risk,n,18.49,...,93.1,66.7,0.0,0.0,550717.0,418934.0,18800.0,369051.0,N,N
6,20400.0,0.2000,540.48,RENT,51315.0,Source Verified,Mar-2019,low_risk,n,25.63,...,100.0,0.0,1.0,0.0,40023.0,23469.0,3500.0,27523.0,N,N
7,10000.0,0.1557,349.46,OWN,66000.0,Source Verified,Mar-2019,low_risk,n,9.00,...,100.0,16.7,0.0,0.0,29300.0,22377.0,8400.0,12500.0,N,N
8,14000.0,0.0881,443.96,RENT,45000.0,Not Verified,Mar-2019,low_risk,n,10.24,...,100.0,0.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0,N,N
9,10000.0,0.1474,345.39,RENT,63295.0,Verified,Mar-2019,low_risk,n,39.34,...,97.2,71.4,0.0,0.0,119822.0,102924.0,15300.0,80722.0,N,N


# Split the Data into Training and Testing

In [19]:
# Create our features
X = pd.get_dummies(df)


# Create our target
y = df["loan_status"]

X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,1,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,1,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,1,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,1,1,1,0,0,1,1,0,1,1


In [20]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.994958,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [21]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

y_train

3903     low_risk
28390    low_risk
15470    low_risk
4279     low_risk
57514    low_risk
           ...   
49100    low_risk
20609    low_risk
21440    low_risk
50057    low_risk
5192     low_risk
Name: loan_status, Length: 51612, dtype: object

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [23]:
# Resample the training data with the RandomOversampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

counter = Counter(y_train)
print ('Before',counter)

ros = RandomOverSampler()
# resampling X, y
X_ros, y_ros = ros.fit_resample(X_train, y_train)

# new loan status distribution 
print('After',Counter(y_ros))


Before Counter({'low_risk': 51366, 'high_risk': 246})
After Counter({'low_risk': 51366, 'high_risk': 51366})


In [24]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(X_ros, y_ros)

y_pred = model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,low_risk,low_risk
1,low_risk,low_risk
2,low_risk,low_risk
3,low_risk,low_risk
4,high_risk,low_risk
5,low_risk,low_risk
6,low_risk,low_risk
7,low_risk,low_risk
8,high_risk,low_risk
9,low_risk,low_risk


In [25]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.6686428363847718


In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[   65    36]
 [ 5665 11439]]


In [27]:
# Print the imbalanced classification report
report = classification_report_imbalanced(y_test, y_pred)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.67      0.02      0.66      0.43       101
   low_risk       1.00      0.67      0.64      0.80      0.66      0.43     17104

avg / total       0.99      0.67      0.64      0.80      0.66      0.43     17205



### SMOTE Oversampling

In [28]:
# Resample the training data with SMOTE
# YOUR CODE HERE

from imblearn.over_sampling import SMOTE
smt = SMOTE()

counter = Counter(y_train)
print ('Before',counter)

# resampling X, y
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

# new distribution 
counter = Counter(y_train_sm)
print ('After',counter)

Before Counter({'low_risk': 51366, 'high_risk': 246})
After Counter({'low_risk': 51366, 'high_risk': 51366})


In [29]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE


from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(X_train_sm, y_train_sm)

y_pred = model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,low_risk,low_risk
3,low_risk,low_risk
4,high_risk,low_risk
5,low_risk,low_risk
6,low_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,high_risk,low_risk


In [30]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
print(accuracy_score(y_test, y_pred))

0.6816623074687591


In [31]:
# Display the confusion matrix
# YOUR CODE HERE
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[   65    36]
 [ 5441 11663]]


In [32]:
# Print the imbalanced classification report
# YOUR CODE HERE
report = classification_report_imbalanced(y_test, y_pred)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.68      0.02      0.66      0.44       101
   low_risk       1.00      0.68      0.64      0.81      0.66      0.44     17104

avg / total       0.99      0.68      0.64      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids 

cc = ClusterCentroids()
# resampling X, y
X_train_cc, y_train_cc = cc.fit_resample(X_train, y_train)

#Print counter before undersampling
counter = Counter(y_train)
print ('Before',counter)

# new loan status distribution 
print(Counter(y_train_cc))


Before Counter({'low_risk': 51366, 'high_risk': 246})
Counter({'high_risk': 246, 'low_risk': 246})


In [34]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

cc.fit(X_train_cc, y_train_cc)

y_pred = model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,low_risk,low_risk
3,low_risk,low_risk
4,high_risk,low_risk
5,low_risk,low_risk
6,low_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,high_risk,low_risk


In [35]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
print(accuracy_score(y_test, y_pred))

0.6816623074687591


In [36]:
# Display the confusion matrix
# YOUR CODE HERE
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[   65    36]
 [ 5441 11663]]


In [37]:
# Print the imbalanced classification report
# YOUR CODE HERE
report = classification_report_imbalanced(y_test, y_pred)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.68      0.02      0.66      0.44       101
   low_risk       1.00      0.68      0.64      0.81      0.66      0.44     17104

avg / total       0.99      0.68      0.64      0.81      0.66      0.44     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [40]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

from imblearn.combine import SMOTEENN

counter = Counter(y_train)
print ('Before',counter)

smt = SMOTEENN()

# resampling X, y
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)

# new loan status distribution 
counter = Counter(y_train_smt)
print ('After',counter)



Before Counter({'low_risk': 51366, 'high_risk': 246})
After Counter({'high_risk': 51361, 'low_risk': 46656})


In [41]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

smt.fit(X_train_smt, y_train_smt)

y_pred = model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)


,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,low_risk,low_risk
3,low_risk,low_risk
4,high_risk,low_risk
5,low_risk,low_risk
6,low_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,high_risk,low_risk


In [42]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
print(accuracy_score(y_test, y_pred))

0.6816623074687591


In [43]:
# Display the confusion matrix
# YOUR CODE HERE

matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[   65    36]
 [ 5441 11663]]


In [44]:
# Print the imbalanced classification report
# YOUR CODE HERE
report = classification_report_imbalanced(y_test, y_pred)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.68      0.02      0.66      0.44       101
   low_risk       1.00      0.68      0.64      0.81      0.66      0.44     17104

avg / total       0.99      0.68      0.64      0.81      0.66      0.44     17205

